# 一、模型融合目标

- 对于多种调参完成的模型进行融合。
- 完成对于多种模型的融合，提交融合结果并打卡。

# 二、内容介绍

模型融合是比赛后期一个重要的环节。大体来说，有如下的类型方式：

1. 简单加权融合：
    - 回归（分类概率）：算术平均融合（Arithmetic Mean），几何平均融合（Geometric Mean）；
    - 分类：投票（Voting）；
    - 综合：排序融合（Rank Averaging），log融合；
2. stacking/blending：
    - 构建多层模型，并利用预测结果再拟合预测。
3. boosting/bagging（在 XGBoost，AdaBoost，GBDT 中已经用到）：
    - 多树的提升方法；

# 三、Stacking 相关理论介绍

## 什么是 stacking

简单来说，stacking 就是当用初始训练数据学习出若干个基学习器后，将这几个学习器的预测结果作为新的训练集，来学习一个新的学习器。

![Concept Diagram of Stacking](http://jupter-oss.oss-cn-hangzhou.aliyuncs.com/public/files/image/2326541042/1584448793231_6TygjXwjNb.jpg)

将个体学习器结合在一起的时候，使用的方法叫做 **结合策略**。对于分类问题，我们可以使用投票法来选择输出最多的类。对于回归问题，我们可以将分类器输出的结果求平均值。

上面说的 **投票法** 和 **平均法** 都是很有效的结合策略。还有一种结合策略是使用另外一个机器学习算法来将个体机器学习器的结果结合在一起，这个方法就是 **stacking**。

在 stacking 方法中，我们把个体学习器叫做 **初级学习器**，用于结合的学习器叫做 **次级学习器** 或 **元学习器（meta-learner）**，次级学习器用于训练的数据叫做 **次级训练集**。次级训练集是在训练集上用初级学习器得到的。

## 如何进行 stacking

算法示意图如下：

![Stacking 算法](http://jupter-oss.oss-cn-hangzhou.aliyuncs.com/public/files/image/2326541042/1584448806789_1ElRtHaacw.jpg)

> 引用自 西瓜书《机器学习》

- 过程 1-3 是训练出来个体学习器，也就是初级学习器。
- 过程 5-9 是使用训练出来的个体学习器得来的预测结果，这个预测的结果当做次级学习器的训练集。
- 过程 11 是用初级学习器预测的结果训练出次级学习器，得到我们最后训练的模型。

## Stacking 的方法讲解

首先，我们先从一种“不那么正确”但是容易懂的 Stacking 方法讲起。

Stacking 模型本质上是一种分层的结构，这里简单起见，只分析二级 Stacking。假设我们有 2 个基模型 Model1_1、Model1_2 和 一个次级模型 Model2。

**Step 1**：基模型 Model1_1，对训练集 train 训练，然后用于预测 train 和 test 的标签列，分别是 P1，T1

Model1_1 模型训练：
$$
\left\{
    \begin{matrix}
        \vdots \\
        X_{train} \\
        \vdots
    \end{matrix}
\right\}
\substack{
    Model1\_1\;Train \\
    \overbrace\Longrightarrow
}
\left\{
    \begin{matrix}
        \vdots \\
        Y_{true} \\
        \vdots
    \end{matrix}
\right\}
$$

训练后的模型 Model1_1 分别在 train 和 test 上预测，得到预测标签分别是 P1, T1
$$
\left\{
    \begin{matrix}
        \vdots \\
        X_{train} \\
        \vdots
    \end{matrix}
\right\}
\substack{
    Model1\_1\;Predict \\
    \overbrace\Longrightarrow
}
\left\{
    \begin{matrix}
        \vdots \\
        P_1 \\
        \vdots
    \end{matrix}
\right\}
$$

$$
\left\{
    \begin{matrix}
        \vdots \\
        X_{test} \\
        \vdots
    \end{matrix}
\right\}
\substack{
    Model1\_1\;Predict \\
    \overbrace\Longrightarrow
}
\left\{
    \begin{matrix}
        \vdots \\
        T_1 \\
        \vdots
    \end{matrix}
\right\}
$$

**Step 2**：基模型 Model1_2，对训练集 train 训练，然后用于预测 train 和 test 的标签列，分别是 P2，T2

Model1_2 模型训练：
$$
\left\{
    \begin{matrix}
        \vdots \\
        X_{train} \\
        \vdots
    \end{matrix}
\right\}
\substack{
    Model1\_2\;Train \\
    \overbrace\Longrightarrow
}
\left\{
    \begin{matrix}
        \vdots \\
        Y_{true} \\
        \vdots
    \end{matrix}
\right\}
$$

训练后的模型 Model1_2 分别在 train 和 test 上预测，得到预测标签分别是 P2，T2
$$
\left\{
    \begin{matrix}
        \vdots \\
        X_{train} \\
        \vdots
    \end{matrix}
\right\}
\substack{
    Model1\_2\;Predict \\
    \overbrace\Longrightarrow
}
\left\{
    \begin{matrix}
        \vdots \\
        P_2 \\
        \vdots
    \end{matrix}
\right\}
$$

$$
\left\{
    \begin{matrix}
        \vdots \\
        X_{test} \\
        \vdots
    \end{matrix}
\right\}
\substack{
    Model1\_2\;Predict \\
    \overbrace\Longrightarrow
}
\left\{
    \begin{matrix}
        \vdots \\
        T_2 \\
        \vdots
    \end{matrix}
\right\}
$$

**Step 3**：分别把 P1，P2 以及 T1，T2 合并，得到一个新的训练集和测试集 train2，test2。
$$
\substack{
    Train_2 \\
    \overbrace{
        \left\{
            \begin{matrix}
                \vdots & \vdots \\
                P1 & P2 \\
                \vdots & \vdots
            \end{matrix}
        \right\}
    }
}
and
\substack{
    Test_2 \\
    \overbrace{
        \left\{
            \begin{matrix}
                \vdots & \vdots \\
                T1 & T2 \\
                \vdots & \vdots
            \end{matrix}
        \right\}
    }
}
$$

再用 次级模型 Model2 以真实训练集标签为 **标签训练**，以 train2 为特征进行训练，预测 test2，得到的测试集预测的标签列 $Y_{Pre}$
$$
\substack{
    Train_2 \\
    \overbrace{
        \left\{
            \begin{matrix}
                \vdots & \vdots \\
                P1 & P2 \\
                \vdots & \vdots
            \end{matrix}
        \right\}
    }
}
\substack{
    Model2\;Train \\
    \overbrace\Longrightarrow
}
\left\{
    \begin{matrix}
        \vdots \\
        Y_{True} \\
        \vdots
    \end{matrix}
\right\}
$$

$$
\substack{
    Test_2 \\
    \overbrace{
        \left\{
            \begin{matrix}
                \vdots & \vdots \\
                P1 & P2 \\
                \vdots & \vdots
            \end{matrix}
        \right\}
    }
}
\substack{
    Model1\_2\;Predict \\
    \overbrace\Longrightarrow
}
\left\{
    \begin{matrix}
        \vdots \\
        Y_{Pre} \\
        \vdots
    \end{matrix}
\right\}
$$

这就是我们两层堆叠的一种基本的原始思路想法。在不同模型预测的结果基础上再加一层模型，进行再训练，从而得到模型最终的预测。

Stacking 本质上就是这么直接的思路，但是直接这样有时对于如果训练集和测试集分布不那么一致的情况下是有一点问题的，器问题在于用初始模型训练的标签再利用真实标签进行再训练，毫无疑问会导致一定的模型过拟合训练集，这样或许模型在测试集上的泛化能力或者说效果会有一定的下降，因此现在的问题变成了如何降低再训练的过拟合性，这里我们一般有两种方法。

- 1. 次级模型尽量选择简单的线性模型
- 2. 利用 K-折交叉验证

K-折交叉验证：

训练：

![训练](http://jupter-oss.oss-cn-hangzhou.aliyuncs.com/public/files/image/2326541042/1584448819632_YvJOXMk02P.jpg)

预测：

![预测](http://jupter-oss.oss-cn-hangzhou.aliyuncs.com/public/files/image/2326541042/1584448826203_k8KPy9n7D9.jpg)

# 四、代码预测

## 回归/分类概率-融合

### 简单加权平均，结果直接融合

In [ ]:
## 生成一些简单的样本数据，test_pre i 代表第i个模型的预测值
test_pre1 = [1.2, 3.2, 2.1, 6.2]
test_pre2 = [0.9, 3.1, 2.0, 5.9]
test_pre3 = [1.1, 2.9, 2.2, 6.0]

# y_test_true 代表第模型的真实值
y_test_true = [1, 3, 2, 6]

In [ ]:
import os
import numpy as np
import pandas as pd

## 定义结果的加权平均函数
def Weighted_method(test_pre1, test_pre2, test_pre3, w=[1/3, 1/3, 1/3]):
    Weighted_result = w[0] * pd.Series(test_pre1) + w[1] * pd.Series(test_pre2) + w[2] * pd.Series(test_pre3)

    return Weighted_result

In [ ]:
from sklearn import metrics

# 各模型的预测结果计算 MAE
print('Pred1 MAE:', metrics.mean_absolute_error(y_test_true, test_pre1))
print('Pred2 MAE:', metrics.mean_absolute_error(y_test_true, test_pre2))
print('Pre3 MAE:', metrics.mean_absolute_error(y_test_true, test_pre3))

In [ ]:
## 根据加权计算 MAE
w = [0.3, 0.4, 0.3]
Weighted_pre = Weighted_method(test_pre1, test_pre2, test_pre3, w)
print('Weighted_pre MAE:', metrics.mean_absolute_error(y_test_true, Weighted_pre))

可以发现，加权结果相对于之前的结果是有提升的，这种我们称其为简单的加权平均。

还有一些特殊的形式，比如 mean 平均，median 平均

In [ ]:
## 定义结果的加权平均函数
def Mean_method(test_pre1, test_pre2, test_pre3):
    Mean_result = pd.concat([
        pd.Series(test_pre1),
        pd.Series(test_pre2),
        pd.Series(test_pre3)
    ], axis=1).mean(axis=1)

    return Mean_result

In [ ]:
Mean_pre = Mean_method(test_pre1, test_pre2, test_pre3)
print('Mean_pre MAE:', metrics.mean_absolute_error(y_test_true, Mean_pre))

In [ ]:
## 定义结果的加权平均函数
def Median_method(test_pre1, test_pre2, test_pre3):
    Median_result = pd.concat([
        pd.Series(test_pre1),
        pd.Series(test_pre2),
        pd.Series(test_pre3)
    ], axis=1).mean(axis=1)

    return Median_result

In [ ]:
Median_pre = Median_method(test_pre1, test_pre2, test_pre3)
print('Median_pre MAE:', metrics.mean_absolute_error(y_test_true, Median_pre))

### Stacking 融合（回归）

In [ ]:
from sklearn import linear_model

def Stacking_method(train_reg1, train_reg2, train_reg3, y_train_true, test_pre1, test_pre3, model_L2=linear_model.LinearRegression()):
    model_L2.fit(
        pd.concat([
            pd.Series(train_reg1),
            pd.Series(train_reg2),
            pd.Series(train_reg3)
        ], axis=1).values,
        y_train_true
    )
    Stacking_result = model_L2.predict(
        pd.concat(
            [
                pd.Series(test_pre1),
                pd.Series(test_pre2),
                pd.Series(test_pre3)
            ],
            axis=1
        ).values
    )

    return Stacking_result

In [ ]:
## 生成一些简单的样本数据，test_prei 代表第i个模型的预测值
train_reg1 = [3.2, 8.2, 9.1, 5.2]
train_reg2 = [2.9, 8.1, 9.0, 4.9]
train_reg3 = [3.1, 7.9, 9.2, 5.0]
# y_test_true 代表第模型的真实值
y_train_true = [3, 8, 9, 5] 

test_pre1 = [1.2, 3.2, 2.1, 6.2]
test_pre2 = [0.9, 3.1, 2.0, 5.9]
test_pre3 = [1.1, 2.9, 2.2, 6.0]

# y_test_true 代表第模型的真实值
y_test_true = [1, 3, 2, 6] 

In [ ]:
model_L2 = linear_model.LinearRegression()
Stacking_pre = Stacking_method(
    train_reg1,
    train_reg2,
    train_reg3,
    y_train_true,
    test_pre1,
    test_pre2,
    test_pre3,
    model_L2
)
print('Stacking_pre MAE:', metrics.mean_absolute_error(y_test_true, Stacking_pre))

可以发现，模型结果相对于之前有进一步提升。我们需要注意的一点是，对于第二层 Stacking 的模型不宜选取的过于复杂，这样会导致模型在训练集上过拟合，从而使得在测试集上并不能达到很好的效果。

## 分类模型融合

对于分类，同样的可以使用融合方法，比如简单投票、Stacking...

In [ ]:
import numpy as np
from sklearn.datasets import make_blobs
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score, roc_auc_score

### Voting 投票机制

Voting 即投票机制，分为 **软投票** 和 **硬投票** 两种，其原理采用少数服从多数的思想。

In [ ]:
"""
硬投票：对多个模型直接进行投票，不区分模型结果的相对重要性，最终投票数最多的类为最终被预测的类。
"""
iris = datasets.load_iris()

x = iris.data
y = iris.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

clf1 = XGBClassifier(learning_rate=0.1, n_estimators=150, max_depth=3, min_child_weight=2, subsample=0.7, colsample_bytree=0.6, objective='binary:logistic')
clf2 = RandomForestClassifier(n_estimators=50, max_depth=1, min_sample_split=4, min_sample_leaf=63, oob_score=True)
clf3 = SVC(C=0.1)

# 硬投票
eclf = VotingClassifier(estimators=[('xgb', clf1), ('rf', clf2), ('svc', clf3)], voting='hard')
for clf, label in zip([clf1, clf2, clf3, eclf], ['XGBBoosting', 'Random Forest', 'SVM', 'Ensemble']):
    scores = cross_val_score(clf, x, y, cv=5, scoring='accuracy')
    print('Accuracy: %0.2f (+/- %0.2f) [%s]' % (scores.mean(), scores.std(), label))

In [ ]:
"""
软投票：和硬投票原理相同，增加了设置权重的功能，可以为不同模型设置不同权重，进而区别模型不同的重要性。
"""

x = iris.data
y = iris.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

clf1 = XGBClassifier(linear_rate=0.1, n_estimators=150, max_depth=3, min_child_weight=2, subsample=0.8, colsample_bytree=0.8, objective='binary:logistic')
clf2 = RandomForestClassifier(n_estimators=50, max_depth=1, min_sample_split=4, min_sample_leaf=63, oob_score=True)

# 软投票
eclf = VotingClassifier(estimators=[('xgb', clf1), ('rf', clf2), ('svc', clf3)], voting='soft', weights=[2, 1, 1])
for clf, label in zip([clf1, clf2, clf3, eclf], ['XGBBoosting', 'Random Forest', 'SVM', 'Ensemble']):
    scores = cross_val_score(clf, x, y, cv=5, scoring='accuracy')
    print('Accuracy: %0.2f (+/- %0.2f) [%s]' % (scores.mean(), scores.std(), label))

### 分类的 Stacking/Blending 融合

stacking 是一种分层模型集成框架。

> 以两层为例，第一层由多个基学习器组成，其输入为原始训练集；第二层的模型则是以第一层基学习器的输出作为训练集进行再训练，从而获取完整的 stacking 模型，stacking 两层模型都使用了全部的训练数据。

In [ ]:
"""
5-Fold Stacking
"""

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
import pandas as pd

# 创建训练的数据集
data_0 = iris.data
data = data_0[:100, :]

target_0 = iris.target
target = target_0[:100]

# 模型融合中使用到的各个单模型
clfs = [
    LogisticRegression(solver='lbfgs'),
    RandomForestClassifier(n_estimators=5, n_jobs=-1, criterion='gini'),
    ExtraTreesClassifier(n_estimators=5, n_jobs=-1, criterion='gini'),
    ExtraTreesClassifier(n_estimators=5, n_jobs=-1, criterion='entropy'),
    GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=5)
]

# 切分一部分数据作为测试集
X, X_predict, y, y_predict = train_test_split(data, target, test_size=0.3, random_state=2020)

dataset_blend_train = np.zores((X.shape[0], len(clfs)))
dataset_blend_test = np.zores((X_predict.shape[0], len(clfs)))

# 5-折 stacking
n_splits = 5
skf = StratifiedKFold(n_splits)
skf = skf.split(X, y)

for j, clf in enumerate(clfs):
    # 依次训练各个单模型
    dataset_blend_test_j = np.zores((X_predict.shape[0], 5))
    for i, (train, test) in enumerate(skf):
        # 5-Fold 交叉训练，使用第 i 个部分作为预测，剩余的部分来训练模型，获得其预测的输出作为第 i 部分的新特征
        X_train, y_train, X_test, y_test = X[train], y[train], X[test], y[test]
        clf.fit(X_train, y_train)
        y_submission = clf.predict_proba(X_test)[:, 1]
        dataset_blend_train[test, j] = y_submission
        dataset_blend_test_j[:, i] = clf.predict_proba(X_predict)[:, 1]
    # 对于测试集，直接用这 k 个模型的预测值均值作为新的特征。
    dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)
    print('Val Auc Score: %f' % roc_auc_score(y_predict, dataset_blend_test[:, j]))

clf = LogisticRegression(solve='lbfgs')
clf.fit(dataset_blend_train, y)
y_submission = clf.predict_proba(dataset_blend_test)[:, 1]

print('Val Auc Score of Stacking: %f' % (roc_auc_score(y_predict, y_submission)))

Blending，其实和 Stacking 是一种类似的多层模型融合的形式。

> 其主要思路是把原始的训练集先分成两部分，比如 70% 的数据作为新的训练集，剩下 30% 的数据作为测试集。

> 在第一层，我们在这 70% 的数据上训练多个模型，然后去预测那 30% 数据的 label，同时也预测测试集的 label。

> 在第二层，我们就直接用这 30% 数据在第一层预测的结果作为新特征继续训练，然后用测试集第一层预测的label做特征，用第二层训练的模型做进一步预测。

其优点在于：

- 比 stacking 简单（因为不用进行 k 次的交叉验证来获得 stacker feature）
- 避开了一个信息泄漏问题：generlizers 和 stacker 使用了不一样的数据集

其缺点在于：

- 使用了很少的数据（第二阶段的只使用 blender 只使用 training set 10% 的量）
- blender 可能会过拟合
- stacking 使用多次的交叉验证会比较稳健

In [ ]:
"""
Blending
"""

# 创建训练的数据集
data_0 = iris.data
data = data_0[:100,:]

target_0 = iris.target
target = target_0[:100]

# 模型融合中使用到的各个单模型
clfs = [
    LogisticRegression(solver='lbfgs'),
    RandomForestClassifier(n_estimators=5, n_jobs=-1, criterion='gini'),
    
]